In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider

N = 100
# Pre-calculate layouts for all possible k values to prevent jumping
# We use a dictionary to store (Graph, Position) pairs
graph_cache = {}
possible_k = range(2, 22, 2)

print("Pre-calculating layouts... please wait.")
for k_val in possible_k:
    G_temp = nx.watts_strogatz_graph(N, k_val, 0.2)
    # Fixed seed in spring_layout ensures the nodes stay in the same spot for a specific k
    pos_temp = nx.spring_layout(G_temp, seed=0) 
    graph_cache[k_val] = (G_temp, pos_temp)

def plot_distribution(n0, n1, n2, n3, k):
    # Retrieve the pre-calculated graph and position
    if k % 2 != 0: k -= 1
    G, pos = graph_cache[k]

    counts = [n0, n1, n2, n3]
    total_assigned = sum(counts)
    if total_assigned > N:
        print(f"Error: Total count ({total_assigned}) exceeds N={N}")
        return

    # 1. ASSIGN THRESHOLDS
    threshold_list = []
    for val, count in enumerate(counts):
        threshold_list.extend([val] * count)
    
    remainder = N - len(threshold_list)
    threshold_list.extend([4] * remainder) 
    
    # Using a fixed seed for shudddfling ensures that if you move a slider 
    # back and forth, the SAME nodes get the SAME thresholds.
    rng = np.random.default_rng(0)
    thresholds = np.array(threshold_list)
    rng.shuffle(thresholds)
    
    labels = {i: (thresholds[i] if thresholds[i] < 5 else '∞') for i in range(N)}
    
    # 2. SIMULATION
    status = ['protesting' if t == 0 else 'quiet' for t in thresholds]
    for _ in range(20):
        changed = False
        new_status = list(status)
        for i in range(N):
            if status[i] == 'quiet':
                neighbors = list(G.neighbors(i))
                protesting_count = sum(1 for nb in neighbors if status[nb] == 'protesting')
                if protesting_count >= thresholds[i]:
                    new_status[i] = 'protesting'
                    changed = True
        status = new_status
        if not changed: break

    # 3. VISUALIZATION
    plt.figure(figsize=(8, 5))
    node_colors = ['#e74c3c' if s == 'protesting' else '#3498db' for s in status]
    
    nx.draw_networkx_edges(G, pos, alpha=0.15, edge_color='gray')
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=150)
    
    # Only draw labels for T=0 to see where the "fire" starts, 
    # or keep them all if N is small enough.
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=7, font_color='white')
    
    plt.title(fr"Connections, k={k} | Protesting: {status.count('protesting')} | Assigned: {total_assigned} / {N}")
    plt.axis('off')
    plt.show()

    N/8
    
# Dashboard
interact(plot_distribution, 
         n0=IntSlider(value=N/8, min=0, max=N/4, description='T=0'), 
         n1=IntSlider(value=N/8, min=0, max=N/4, description='T=1'),
         n2=IntSlider(value=N/8, min=0, max=N/4, description='T=2'),
         n3=IntSlider(value=N/8, min=0, max=N/4, description='T=3'),
         k=IntSlider(value=4, min=2, max=20, step=1, description='Connections'))

Pre-calculating layouts... please wait.


interactive(children=(IntSlider(value=12, description='T=0', max=25), IntSlider(value=12, description='T=1', m…

<function __main__.plot_distribution(n0, n1, n2, n3, k)>

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider

N = 100
graph_cache = {}
for k_val in [4, 6]:
    G_temp = nx.watts_strogatz_graph(N, k_val, 0.2)
    pos_temp = nx.spring_layout(G_temp, seed=0) 
    graph_cache[k_val] = (G_temp, pos_temp)

def run_sim(G, thresholds):
    status = ['protesting' if t == 0 else 'quiet' for t in thresholds]
    for _ in range(20):
        changed, new_status = False, list(status)
        for i in range(N):
            if status[i] == 'quiet' and sum(1 for nb in G.neighbors(i) if status[nb] == 'protesting') >= thresholds[i]:
                new_status[i] = 'protesting'
                changed = True
        status = new_status
        if not changed: break
    return status.count('protesting')

def plot_distribution(n1, n2):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    rng = np.random.default_rng(0)
    n1_range = np.arange(0, 51, 4) # Slightly larger step for speed with averaging
    num_runs = 15 # Number of simulations to average

    scenarios = [
        (4, 10, '-', '#3498db'), # k=4, t0=10 (Solid Blue)
        (4, 20, '-', '#e74c3c'), # k=4, t0=20 (Solid Red)
        (6, 10, '--', '#2980b9'), # k=6, t0=10 (Dashed Dark Blue)
        (6, 20, '--', '#c0392b')  # k=6, t0=20 (Dashed Dark Red)
    ]

    # --- 1. RUN AVERAGED SCENARIOS ---
    for k_val, t0_val, lstyle, col in scenarios:
        G, _ = graph_cache[k_val]
        avg_outcomes = []
        
        for test_n1 in n1_range:
            run_results = []
            for _ in range(num_runs):
                t_list = []
                for val, count in enumerate([t0_val, test_n1, n2]): 
                    t_list.extend([val] * count)
                t_list.extend([4] * max(0, (N - len(t_list))))
                test_thresholds = np.array(t_list[:N])
                rng.shuffle(test_thresholds)
                run_results.append(run_sim(G, test_thresholds))
            avg_outcomes.append(np.mean(run_results))
        
        ax2.plot(n1_range, avg_outcomes, label=f"k={k_val}, T0={t0_val}", 
                 color=col, linestyle=lstyle, linewidth=2.5)

    # --- 2. LIVE VIEW (k=4, T0=10) ---
    G_viz, pos_viz = graph_cache[4]
    t_list_viz = []
    for val, count in enumerate([10, n1, n2]): t_list_viz.extend([val] * count)
    t_list_viz.extend([4] * max(0, (N - len(t_list_viz))))
    viz_thresholds = np.array(t_list_viz[:N])
    rng.shuffle(viz_thresholds)
    
    status = ['protesting' if t == 0 else 'quiet' for t in viz_thresholds]
    for _ in range(20):
        changed, new_status = False, list(status)
        for i in range(N):
            if status[i] == 'quiet' and sum(1 for nb in G_viz.neighbors(i) if status[nb] == 'protesting') >= viz_thresholds[i]:
                new_status[i] = 'protesting'
                changed = True
        status = new_status
        if not changed: break
        
    colors = ['#e74c3c' if s == 'protesting' else '#3498db' for s in status]
    labels = {i: (viz_thresholds[i] if viz_thresholds[i] < 4 else '∞') for i in range(N)}

    nx.draw(G_viz, pos_viz, ax=ax1, node_color=colors, node_size=250, alpha=0.8, edge_color='gray', width=0.4)
    nx.draw_networkx_labels(G_viz, pos_viz, labels=labels, ax=ax1, font_size=8, font_color='white')
    ax1.set_title(f"Live Snapshot (k=4, T0=10)\nActive Nodes: {status.count('protesting')}")

    # --- 3. STYLING ---
    ax2.axvline(x=n1, color='black', linestyle=':', alpha=0.4, label='Current n1')
    ax2.set_xlabel("Number of T=1 Agents (Bridges)")
    ax2.set_ylabel("Mean Final Active Fraction")
    ax2.set_title(f"Ensemble Average (n={num_runs} runs per point)")
    ax2.legend(title="Network Scenarios", loc='upper left')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

interact(plot_distribution, 
         n1=IntSlider(value=10, min=0, max=50, description='T=1'),
         n2=IntSlider(value=15, min=0, max=50, description='T=2'))

interactive(children=(IntSlider(value=10, description='T=1', max=50), IntSlider(value=15, description='T=2', m…

<function __main__.plot_distribution(n1, n2)>